In [1]:
!wget https://motchallenge.net/data/MOT17.zip


--2024-07-04 07:42:11--  https://motchallenge.net/data/MOT17.zip
Resolving motchallenge.net (motchallenge.net)... 131.159.19.34, 2a09:80c0:18::1034
Connecting to motchallenge.net (motchallenge.net)|131.159.19.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5860214001 (5.5G) [application/zip]
Saving to: 'MOT17.zip'

MOT17.zip           100%[===================>]   5.46G  30.3MB/s    in 2m 51s  

2024-07-04 07:45:03 (32.7 MB/s) - 'MOT17.zip' saved [5860214001/5860214001]



In [2]:
!unzip -qq MOT17.zip

In [3]:
import os
import shutil

In [4]:
testdir = 'MOT17/test'
traindir = 'MOT17/train'


In [5]:
!cd ..
os.makedirs('MOT17/test/labels', exist_ok=True)
os.makedirs('MOT17/test/images', exist_ok=True)

In [6]:
import pandas as pd
import os
import yaml
import shutil
import configparser

from tqdm import tqdm

In [7]:
def convert_to_yolo_format(bb, img_w, img_h):
    x_center = bb['bb_left'] + (bb['bb_width'] / 2)
    y_center = bb['bb_top'] + (bb['bb_height'] / 2) 
    
    x_center /= img_w
    y_center /= img_h

    bb_width_normalized = bb['bb_width'] / img_w
    bb_height_normalized = bb['bb_height'] / img_h

    x_center = max(min(x_center, 1.0), 0.0)
    y_center = max(min(y_center, 1.0), 0.0)

    bb_width_normalized = max(min(bb_width_normalized, 1.0), 0.0)
    bb_height_normalized = max(min(bb_height_normalized, 1.0), 0.0)

    return x_center, y_center, bb_width_normalized, bb_height_normalized

In [8]:
def process_folder(folder_path):
    config = configparser.ConfigParser()
    config.read(os.path.join(folder_path, 'seqinfo.ini'))
    img_w = int(config['Sequence']['imWidth'])
    img_h = int(config['Sequence']['imHeight'])

    gt_path = os.path.join(folder_path, 'det/det.txt')
    gt_data = pd.read_csv(gt_path, sep=',', header=None,
                          names=['frame', 'id', 'bb_left', 'bb_top', 'bb_width', 'bb_height', 'conf', 'class', 'visibility'])

    labels_path = os.path.join(folder_path, 'labels')
    os.makedirs(labels_path, exist_ok=True)

    images_path = os.path.join(folder_path, 'images')
    os.makedirs(images_path, exist_ok=True)

    for frame_number in gt_data['frame'].unique():
        frame_data = gt_data[gt_data['frame'] == frame_number]
        label_file = os.path.join(labels_path, f'{frame_number:06d}.txt')

        with open(label_file, 'w') as f:
            for _, row in frame_data.iterrows():
                yolo_bb = convert_to_yolo_format(row, img_w, img_h)
                f.write(f'0 {yolo_bb[0]} {yolo_bb[1]} {yolo_bb[2]} {yolo_bb[3]}\n')

In [9]:
def process_all_folders(base_dir):
    for folder_name in tqdm(os.listdir(base_dir)):
        folder_path = os.path.join(base_dir, folder_name)

        if 'FRCNN' not in folder_name:
            os.system(f'rm -rf {folder_path}')
            continue

        if os.path.isdir(folder_path):
            process_folder(folder_path)



In [10]:
process_all_folders('MOT17/train')
process_all_folders('MOT17/test')

100%|██████████| 23/23 [00:13<00:00,  1.67it/s]


In [11]:
def rename_and_move_files(src_folder, dst_folder,
                          folder_name, file_extension):
    for filename in os.listdir(src_folder):
        if filename.endswith(file_extension):
            new_filename = f'{folder_name}_{filename}'
            shutil.move(os.path.join(src_folder, filename),
                        os.path.join(dst_folder, new_filename))

In [12]:
def move_files_all_folders(base_dir):
    image_dir = os.path.join(base_dir, 'images')
    label_dir = os.path.join(base_dir, 'labels')
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(label_dir, exist_ok=True)

    for folder_name in tqdm(os.listdir(base_dir)):
        if folder_name in ['images', 'labels']:
            continue

        folder_path = os.path.join(base_dir, folder_name)
        if os.path.isdir(folder_path):
            rename_and_move_files(os.path.join(folder_path, 'img1'),
                                  image_dir, folder_name, '.jpg')
            rename_and_move_files(os.path.join(folder_path, 'labels'),
                                  label_dir, folder_name, '.txt')



In [13]:
move_files_all_folders('MOT17/train')
move_files_all_folders('MOT17/test')


100%|██████████| 9/9 [00:00<00:00, 22.52it/s]


In [14]:
def delete_subfolders(base_dir):
    for folder_name in tqdm(os.listdir(base_dir)):
        folder_path = os.path.join(base_dir, folder_name)
        if os.path.isdir(folder_path) and folder_name not in ['images', 'labels']:
            shutil.rmtree(folder_path)
            print(f"Deleted folder: {folder_name}")

In [15]:
delete_subfolders('MOT17/train')
delete_subfolders('MOT17/test')

100%|██████████| 9/9 [00:00<00:00, 1519.00it/s]


Deleted folder: MOT17-02-FRCNN
Deleted folder: MOT17-11-FRCNN
Deleted folder: MOT17-13-FRCNN
Deleted folder: MOT17-09-FRCNN
Deleted folder: MOT17-10-FRCNN
Deleted folder: MOT17-05-FRCNN
Deleted folder: MOT17-04-FRCNN


100%|██████████| 9/9 [00:00<00:00, 1660.89it/s]

Deleted folder: MOT17-01-FRCNN
Deleted folder: MOT17-12-FRCNN
Deleted folder: MOT17-08-FRCNN
Deleted folder: MOT17-06-FRCNN
Deleted folder: MOT17-07-FRCNN
Deleted folder: MOT17-03-FRCNN
Deleted folder: MOT17-14-FRCNN


In [16]:
class_labels = ['object']
dataset_root_dir = os.path.join(os.getcwd(), 'MOT17')
yolo_yaml_path = os.path.join(dataset_root_dir, 'mot17_data.yaml')
data_yaml = {
    'path' : dataset_root_dir,
    'train' : 'train/images',
    'val' : 'test/images',
    'nc' : len(class_labels),
    'names' : class_labels
}

with open(yolo_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

In [17]:
!pip install ultralytics -q

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.48 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5689.3/8062.4 GB disk)


In [18]:
from ultralytics import YOLO

In [19]:
model = YOLO('yolov8n.pt')

epoch = 30
batch_size = -1
img_size = 640
project_name = 'models/yolo'
name = 'yolov8s_mot17_det'

result = model.train(
    data=yolo_yaml_path,
    epochs=epoch,
    batch=batch_size,
    imgsz=img_size,
    project=project_name,
    name=name)

100%|██████████| 6.25M/6.25M [00:00<00:00, 167MB/s]

Ultralytics YOLOv8.2.48 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/MOT17/mot17_data.yaml, epochs=30, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=models/yolo, name=yolov8s_mot17_det, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tru

100%|██████████| 755k/755k [00:00<00:00, 29.8MB/s]
2024-07-04 07:47:07,806	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-07-04 07:47:08,740	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/working/MOT17/train/labels... 5316 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5316/5316 [00:04<00:00, 1089.86it/s]


...

                   all       5919     110141      0.905      0.789      0.892      0.625

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      30/30      9.56G     0.5999      0.346     0.8268        797        640: 100%|██████████| 74/74 [01:05<00:00,  1.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:41<00:00,  1.02it/s]


                   all       5919     110141      0.899      0.788      0.888       0.62

30 epochs completed in 0.941 hours.
Optimizer stripped from models/yolo/yolov8s_mot17_det/weights/last.pt, 6.2MB
Optimizer stripped from models/yolo/yolov8s_mot17_det/weights/best.pt, 6.2MB

Validating models/yolo/yolov8s_mot17_det/weights/best.pt...
Ultralytics YOLOv8.2.48 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [01:01<00:00,  1.45s/it]


                   all       5919     110141      0.896      0.795      0.892      0.626
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to models/yolo/yolov8s_mot17_det
